# 512hw2 Bias

In [128]:
# Import the libraries
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1 - data acquisition
There are two initial data files downloaded in csv format.

## Step 2 - data cleaning
filter the page_data by removing the page names that start with the string 'Template' in page_data.csv

In [155]:
page_data = pd.read_csv('page_data.csv')
WPDS_data = pd.read_csv('WPDS_2020_data.csv')

In [156]:
print(page_data.shape)

(47197, 3)


In [157]:
page_data_cleaned = page_data.loc[(page_data['page'].apply(lambda x: (x[0:8] != 'Template')))]
print(page_data_cleaned.shape)
page_data_cleaned

(46701, 3)


,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


In [158]:
WPDS_data

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000
230,SB,Solomon Islands,Country,2019,0.715,715000
231,TO,Tonga,Country,2019,0.099,99000
232,TV,Tuvalu,Country,2019,0.010,10000


In [159]:
WPDS_data_reduced = WPDS_data[['Name', 'Type', 'Population']]
WPDS_data_reduced

,Name,Type,Population
0,WORLD,World,7772850000
1,AFRICA,Sub-Region,1337918000
2,NORTHERN AFRICA,Sub-Region,244344000
3,Algeria,Country,44357000
4,Egypt,Country,100803000
...,...,...,...
229,Samoa,Country,200000
230,Solomon Islands,Country,715000
231,Tonga,Country,99000
232,Tuvalu,Country,10000


## Step 3 - get article quality prediction 
I chose to use REST API endpoint to get the prediction results

In [47]:
# api header
headers = {
    'User-Agent': 'https://github.com/tommycqy',
    'From': 'qingyuc@uw.edu'
}

# api endpoint
endpoint = "https://ores.wikimedia.org/v3/scores/enwiki?models=articlequality&revids={rev_ids}"

In [48]:
# 
def api_call(endpoint, rev_ids):
    call = requests.get(endpoint.format(rev_ids = rev_ids), headers=headers)
    response = call.json()
    
    return response

In [85]:
def call_api(score_dict):
    for i in range(0, len(page_data_cleaned), 50):
        if i + 50 > len(page_data_cleaned):
            batch_ids = page_data_cleaned.rev_id.iloc[i:]
        else:
            batch_ids = page_data_cleaned.rev_id.iloc[i:i+50]
        res = api_call(endpoint, '|'.join(str(s) for s in batch_ids))
        for key in res['enwiki']['scores']:
            if 'score' in res['enwiki']['scores'][key]['articlequality']:
                curr_score = res['enwiki']['scores'][key]['articlequality']['score']['prediction']
                score_dict[key] = curr_score
            else:
                score_dict[key] = 'NA'

In [86]:
score_dict = dict()
call_api(score_dict)

In [143]:
print(len(score_dict))

46701


After we got all the predictions (some of them are NAs), we need to filter out the NAs and keep a seperate log.

In [201]:
page_data_cleaned['article_quality_est.'] = page_data_cleaned['rev_id'].astype(str).map(score_dict)
no_score_log = page_data_cleaned.loc[page_data_cleaned['article_quality_est.']=='NA']
print(len(no_score_log))
no_score_log


276


<ipython-input-201-fdffb979ed9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  page_data_cleaned['article_quality_est.'] = page_data_cleaned['rev_id'].astype(str).map(score_dict)


,page,country,rev_id,article_quality_est.
126,List of politicians in Poland,Poland,516633096,NA
222,Tingtingru,Vanuatu,550682925,NA
330,Daud Arsala,Afghanistan,627547024,NA
359,Book:Two Political Biographies,India,636911471,NA
514,Dilaver Bey,Turkey,669987106,NA
...,...,...,...,...
46782,John Rose (Trotskyist),United Kingdom,807336308,NA
46862,Jalal Movaghar,Iran,807367030,NA
46863,Mohsen Movaghar,Iran,807367166,NA
47182,King Gutierrez,Philippines,807479587,NA


In [202]:
page_data_reduced = page_data_cleaned.loc[page_data_cleaned['article_quality_est.']!='NA']
print(len(page_data_reduced))

46425


## Step 4 - combine the datasets 
Remove any rows that do not have matching data, and output them to a CSV file called: wp_wpds_countries-no_match.csv

Consolidate the remaining data into a single CSV file called: wp_wpds_politicians_by_country.csv


In [203]:
df_merge = page_data_reduced.merge(WPDS_data_reduced,how='outer',left_on=['country'],right_on=['Name'])
print(len(df_merge))

46476


In [204]:
no_match_df = df_merge.loc[df_merge['country'].isna() | df_merge['Name'].isna()]
no_match_index = no_match_df.index
no_match_df

,page,country,rev_id,article_quality_est.,Name,Type,Population
488,Julius Gregr,Czech Republic,395521877.0,Stub,NaN,NaN,NaN
489,Edvard Gregr,Czech Republic,395526568.0,Stub,NaN,NaN,NaN
490,Miroslav Poche,Czech Republic,672862914.0,Stub,NaN,NaN,NaN
491,Vojtěch Mynář,Czech Republic,673008587.0,Stub,NaN,NaN,NaN
492,Jan Malypetr,Czech Republic,704424304.0,Stub,NaN,NaN,NaN
...,...,...,...,...,...,...,...
46471,NaN,NaN,NaN,NaN,French Polynesia,Country,280000.0
46472,NaN,NaN,NaN,NaN,Guam,Country,175000.0
46473,NaN,NaN,NaN,NaN,New Caledonia,Country,295000.0
46474,NaN,NaN,NaN,NaN,Palau,Country,18000.0


In [209]:
no_match_df.to_csv('wp_wpds_countries-no_match.csv', index=False)  

In [208]:
filter_df  = df_merge[~df_merge.index.isin(no_match_index)]
filter_df.rev_id = filter_df.rev_id.astype('int64')
filter_df.Population = filter_df.Population.astype('int64')
filter_df_reduced = filter_df[['page','country', 'rev_id', 'article_quality_est.','Population']]
filter_df_reduced.rename(columns={'page':'article_name','rev_id':'revision_id','Population':'population'}, inplace=True)
filter_df_reduced

C:\Users\Qingyu Chen\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\Qingyu Chen\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,article_name,country,revision_id,article_quality_est.,population
0,Bir I of Kanem,Chad,355319463,Stub,16877000
1,Abdullah II of Kanem,Chad,498683267,Stub,16877000
2,Salmama II of Kanem,Chad,565745353,Stub,16877000
3,Kuri I of Kanem,Chad,565745365,Stub,16877000
4,Mohammed I of Kanem,Chad,565745375,Stub,16877000
...,...,...,...,...,...
46414,Rita Sinon,Seychelles,800323154,Stub,98000
46415,Sylvette Frichot,Seychelles,800323798,Stub,98000
46416,May De Silva,Seychelles,800969960,Start,98000
46417,Vincent Meriton,Seychelles,802051093,Stub,98000


In [210]:
filter_df_reduced.to_csv('wp_wpds_politicians_by_country.csv', index=False)  

## Step 5 - analysis & results
There are six analysis tables below, corresponding to the six analysis questions


First Analysis Task: Top 10 countries by coverage: 10 highest-ranked countries in terms of number of politician articles as a proportion of country population

In [239]:
article_count = filter_df_reduced.groupby('country').agg({'article_name':'count'})

population = filter_df_reduced.groupby('country').agg({'population':'mean'})

percentage = article_count['article_name'].astype('int64') / population['population'].astype('int64') * 100

print(percentage.sort_values(ascending=False).head(10))

country
Tuvalu                            0.540000
Nauru                             0.472727
San Marino                        0.238235
Monaco                            0.105263
Liechtenstein                     0.071795
Marshall Islands                  0.064912
Tonga                             0.063636
Iceland                           0.054620
Andorra                           0.041463
Federated States of Micronesia    0.033962
dtype: float64


Second Analysis Task: Bottom 10 countries by coverage: 10 lowest-ranked countries in terms of number of politician articles as a proportion of country population

In [241]:
print(percentage.sort_values(ascending=True).head(10))

country
India           0.000069
Indonesia       0.000077
China           0.000081
Uzbekistan      0.000082
Ethiopia        0.000088
Zambia          0.000136
Korea, North    0.000140
Thailand        0.000168
Mozambique      0.000186
Bangladesh      0.000187
dtype: float64


Third Analysis Task: Top 10 countries by relative quality: 10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [320]:
groups = filter_df_reduced.groupby('country')
group = groups.apply(lambda g: g[(g['article_quality_est.'] == 'FA') | (g['article_quality_est.'] == 'GA')])

group

article_name      country  revision_id  \
country                                                                     
Afghanistan 31260             Jan Mohammad Khan  Afghanistan    794478106   
            31286           Shah Shujah Durrani  Afghanistan    796493730   
            31287     Abdul Haq (Afghan leader)  Afghanistan    796569814   
            31299  Nasrullah Khan (Afghanistan)  Afghanistan    797155187   
            31316                Amrullah Saleh  Afghanistan    799215547   
...                                         ...          ...          ...   
Yemen       28451           Ali Mohsen al-Ahmar        Yemen    790827026   
            28486               Ahmad bin Yahya        Yemen    802588416   
            28492               Tawakkol Karman        Yemen    805749133   
Zimbabwe    21177                 Robert Mugabe     Zimbabwe    806814411   
            21179                  Grace Mugabe     Zimbabwe    807337237   

                  article_quality_est.  population  
country                                             
Afghanistan 31260                   GA    38928000  
            31286                   GA    38928000  
            31287                   GA    38928000  
            31299                   GA    38928000  
            31316                   FA    38928000  
...                                ...         ...  
Yemen       28451                   GA    29826000  
            28486                   GA    29826000  
            28492                   FA    29826000  
Zimbabwe    21177                   FA    14863000  
            21179                   GA    14863000  

[1028 rows x 5 columns]

In [326]:
high_quality_article_count = group.groupby(level=[0]).size()

country_with_hqa_population = group.groupby(level=[0]).agg({'population':'mean'})

In [336]:
hqa_proportion_by_country = high_quality_article_count / country_with_hqa_population['population'].astype('int64') * 100
print(hqa_proportion_by_country.sort_values(ascending=False).head(10))

country
Tuvalu         0.040000
Dominica       0.001389
Vanuatu        0.000935
Iceland        0.000543
Ireland        0.000500
Montenegro     0.000322
Martinique     0.000281
Bhutan         0.000274
New Zealand    0.000261
Romania        0.000218
dtype: float64


Forth Analysis Task: Bottom 10 countries by relative quality: 10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [337]:
print(hqa_proportion_by_country.sort_values(ascending=True).head(10))

country
India         9.285051e-07
Nigeria       9.702144e-07
Tanzania      1.674088e-06
Ethiopia      1.740402e-06
Bangladesh    1.766691e-06
Colombia      2.022490e-06
Uganda        2.186222e-06
Morocco       2.781486e-06
Brazil        2.832701e-06
China         2.852284e-06
dtype: float64


Fifth Analysis Task: Geographic regions by coverage: Ranking of geographic regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population

Sixth Analysis Task: Geographic regions by coverage: Ranking of geographic regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality